<a href="https://colab.research.google.com/github/Noonanav/phage_host_set_transformers/blob/main/set_transformer_modeling_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Transformer for Strain-Phage Interaction Prediction

## Block 1: Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
import warnings
from tqdm import tqdm
import time
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import seaborn as sns
import pickle
import os
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Block 2: Data Loading and Preprocessing
Description: This block handles the loading of strain and phage embeddings, splitting the interactions into train and test sets, creating datasets and data loaders, and managing variable-length inputs.

## Hyperparameters and Architectural Decisions
### `filter_interactions_by_strain` function:
- `test_size`: Proportion of strains used for testing.
  - Current value: 0.2
  - Possible values to try: 0.1, 0.15, 0.25, 0.3

### `create_data_loaders` function:  
- `batch_size`: Number of samples per batch during training and testing.
  - Current value: 1
  - Possible values to try: 2, 4, 8, 16, 32
- `num_workers`: Number of parallel workers for data loading.
  - Current value: 0
  - Possible values to try: 2, 4, 8

In [3]:
from pathlib import Path

def load_embeddings(embedding_dir: str) -> Dict[str, np.ndarray]:
    """
    Load all embeddings from a directory (.npy files).
    Each file should contain shape [num_genes, 1280].
    """
    embedding_dir = Path(embedding_dir)
    embeddings = {}

    for file_path in embedding_dir.glob('*.npy'):
        identifier = file_path.stem  # filename without extension
        print('Parsing: ', identifier)
        embedding = np.load(file_path)
        embeddings[identifier] = embedding

    print(f"Loaded {len(embeddings)} embeddings from {embedding_dir}")
    return embeddings

def filter_interactions_by_strain(interactions_df: pd.DataFrame,
                                  random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split interactions into train/test by strain.
    """
    unique_strains = interactions_df['strain'].unique()
    train_strains, test_strains = train_test_split(
        unique_strains,
        test_size=0.2,
        random_state=random_state
    )

    train_df = interactions_df[interactions_df['strain'].isin(train_strains)]
    test_df = interactions_df[interactions_df['strain'].isin(test_strains)]

    print(f"Train set: {len(train_df)} interactions, {len(train_strains)} strains")
    print(f"Test set:  {len(test_df)} interactions, {len(test_strains)} strains")
    return train_df, test_df

class StrainPhageDataset(Dataset):
    def __init__(self,
                 interactions_df: pd.DataFrame,
                 strain_embeddings: Dict[str, np.ndarray],
                 phage_embeddings: Dict[str, np.ndarray]):
        """
        Dataset for strain-phage interactions.
        interactions_df has columns: ['strain', 'phage', 'interaction'].
        """
        self.interactions = interactions_df.reset_index(drop=True)
        self.strain_embeddings = strain_embeddings
        self.phage_embeddings = phage_embeddings

        # Check for missing embeddings
        missing_strains = set(self.interactions['strain']) - set(strain_embeddings.keys())
        missing_phages = set(self.interactions['phage']) - set(phage_embeddings.keys())

        if missing_strains or missing_phages:
            raise ValueError(
                f"Missing embeddings for {len(missing_strains)} strains "
                f"and {len(missing_phages)} phages."
            )

    def __len__(self):
        return len(self.interactions)

    def __getitem__(self, idx):
        row = self.interactions.iloc[idx]
        strain_emb = torch.tensor(
            self.strain_embeddings[row['strain']], dtype=torch.float32
        )  # shape [n_s, 1280]
        phage_emb = torch.tensor(
            self.phage_embeddings[row['phage']], dtype=torch.float32
        )   # shape [n_p, 1280]
        label = torch.tensor(row['interaction'], dtype=torch.float32)
        return strain_emb, phage_emb, label

def collate_variable_sets(batch):
    """
    Collate function to simply stack each list of embeddings
    into a batch dimension.
    NOTE: If the sets truly vary in length,
    torch.stack(...) will fail unless they match or you pad them.
    For now, we assume the user can handle variable-size
    embeddings at runtime or that all sets in a single batch
    have consistent shapes.
    """
    strains, phages, labels = zip(*batch)
    # Each item in 'strains' is shape [n_s, 1280], etc.

    # If all n_s differ, stacking might fail.
    # For a quick fix, you could feed batch_size=1
    # or do custom padding.
    # We'll assume they can be stacked.
    return (
        torch.stack(strains, dim=0),
        torch.stack(phages, dim=0),
        torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Reshape to [B, 1]
    )

def create_data_loaders(train_df: Optional[pd.DataFrame],
                            test_df: pd.DataFrame,
                            strain_embeddings: Dict[str, np.ndarray],
                            phage_embeddings: Dict[str, np.ndarray],
                            batch_size: int = 1) -> Tuple[DataLoader, DataLoader]:

        train_loader = None # Initialize train_loader as None
        if train_df is not None and not train_df.empty: # add a check before proceeding:
            train_dataset = StrainPhageDataset(train_df, strain_embeddings, phage_embeddings)
            train_loader = DataLoader(
                train_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0,
                pin_memory=True,
                collate_fn=collate_variable_sets
            )

        test_dataset  = StrainPhageDataset(test_df, strain_embeddings, phage_embeddings)
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True,
            collate_fn=collate_variable_sets
        )

        return train_loader, test_loader # now, it can return None for `train_loader`

# Block 3: Set Transformer Blocks
Description: This block implements the core components of the Set Transformer architecture, including the Multi-head Attention Block (MAB), Induced Set Attention Block (ISAB), and Pooling by Multihead Attention (PMA).

## Hyperparameters and Architectural Decisions
### `MAB` class:
- `num_heads`: Number of attention heads in the multi-head attention mechanism.
  - Current value: Determined by the `num_heads` parameter passed to the `ISAB` and `PMA` classes
  - Possible values to try: 4, 8, 12, 16
- `ln`: Whether to apply layer normalization after the attention operation.
  - Current value: False
  - Possible values to try: True

### `ISAB` class:
- `num_inds`: Number of inducing points used to reduce computational complexity.
  - Current value: Determined by the `strain_inds` and `phage_inds` parameters in the `StrainPhageTransformer` class
    - For the strain encoder: `strain_inds=128`
    - For the phage encoder: `phage_inds=64`
  - Possible values to try: 32, 64, 96, 128
- `ln`: Whether to apply layer normalization.
  - Current value: False
  - Possible values to try: True

### `PMA` class:
- `num_heads`: Number of attention heads in the pooling mechanism.
  - Current value: Determined by the `num_heads` parameter passed to the `StrainPhageTransformer` class
  - Possible values to try: 4, 8, 12, 16  
- `num_seeds`: Number of output vectors after pooling.
  - Current value: 1
  - Possible values to try: 2, 4, 8
- `ln`: Whether to apply layer normalization.  
  - Current value: False
  - Possible values to try: True

In [4]:
class MAB(nn.Module):
    """Multi-head Attention Block."""
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        self.fc_o = nn.Linear(dim_V, dim_V)
        self.ln = nn.LayerNorm(dim_V) if ln else nn.Identity()

    def forward(self, Q, K, return_attn=False):
        Q_ = self.fc_q(Q)
        K_ = self.fc_k(K)
        V_ = self.fc_v(K)

        dim_split = self.dim_V // self.num_heads
        B, n_q, _ = Q_.shape
        n_k = K_.size(1)

        Q_ = Q_.view(B, n_q, self.num_heads, dim_split).transpose(1, 2)
        K_ = K_.view(B, n_k, self.num_heads, dim_split).transpose(1, 2)
        V_ = V_.view(B, n_k, self.num_heads, dim_split).transpose(1, 2)

        A = torch.matmul(Q_, K_.transpose(-2, -1)) / np.sqrt(dim_split)
        A = torch.softmax(A, dim=-1)  # [B, h, n_q, n_k]

        O = torch.matmul(A, V_)  # [B, h, n_q, d_h]
        O = O.transpose(1, 2).contiguous().view(B, n_q, self.dim_V)
        O = self.ln(O + F.relu(self.fc_o(O)))

        if return_attn:
            # Return both output and attention
            return O, A
        else:
            return O

class ISAB(nn.Module):
    """
    Induced Set Attention Block.
    Reduces complexity by using M learned 'inducing' vectors.
    """
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False):
        super(ISAB, self).__init__()
        self.num_inds = num_inds
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)

        self.mab1 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln)
        self.mab2 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln)

    def forward(self, X):
        # 1) Inducing points attend to input
        #    I ~ [1, M, d_out], repeat to [B, M, d_out]
        H = self.mab1(self.I.repeat(X.size(0),1,1), X)  # [B, M, d_out]
        # 2) Input attends back to induced
        H = self.mab2(X, H)                             # [B, n, d_out]
        return H

class PMA(nn.Module):
    """
    Pooling by Multihead Attention to reduce a set to num_seeds outputs.
    By default, set num_seeds=1 to get a single vector.
    """
    def __init__(self, dim, num_heads, num_seeds=1, ln=False):
        super(PMA, self).__init__()
        self.num_seeds = num_seeds
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB(dim, dim, dim, num_heads, ln=ln)

    def forward(self, X):
        # X: [B, n, d]
        # seeds: [1, num_seeds, d]
        B = X.size(0)
        S = self.S.repeat(B, 1, 1)  # [B, num_seeds, d]
        return self.mab(S, X)       # [B, num_seeds, d]

# Block 4: Cross-Attention  
Description: This block implements the cross-attention mechanism, allowing strain embeddings to attend to phage embeddings and vice versa.

## Hyperparameters and Architectural Decisions
### `CrossAttention` class:
- `num_heads`: Number of attention heads in the cross-attention mechanism.
  - Current value: Determined by the `num_heads` parameter passed to the `StrainPhageTransformer` class
  - Possible values to try: 4, 8, 12, 16
- `ln`: Whether to apply layer normalization.
  - Current value: False
  - Possible values to try: True

In [5]:
class CrossAttention(nn.Module):
    """Cross-attention between two sets."""
    def __init__(self, dim, num_heads, ln=False):
        super(CrossAttention, self).__init__()
        self.mab = MAB(dim, dim, dim, num_heads, ln=ln)

    def forward(self, X, Y, return_attn=False):
        # Pass the return_attn flag to MAB
        return self.mab(X, Y, return_attn=return_attn)

# Block 5: Encoders and Complete Model
Description: This block defines the set encoders for strain and phage embeddings and the complete `StrainPhageTransformer` model, which combines the set encoders, cross-attention blocks, pooling, and classification layers.

## Hyperparameters and Architectural Decisions
### `SetEncoder` class:
- `num_heads`: Number of attention heads in the ISAB layers.
  - Current value: Determined by the `num_heads` parameter passed to the `StrainPhageTransformer` class
  - Possible values to try: 4, 8, 12, 16
- `num_inds`: Number of inducing points used in the ISAB layers.
  - Current value: Determined by the `strain_inds` and `phage_inds` parameters in the `StrainPhageTransformer` class
    - For the strain encoder: `strain_inds=128`
    - For the phage encoder: `phage_inds=64`
  - Possible values to try: 32, 64, 96, 128
- `ln`: Whether to apply layer normalization.
  - Current value: False
  - Possible values to try: True

### `StrainPhageTransformer` class:
- `embedding_dim`: Dimension of the input embeddings.
  - Current value: 1280
  - Possible values to try: 512, 768, 1024 (depending on the embedding model)
- `num_heads`: Number of attention heads in the cross-attention blocks and PMA layers.
  - Current value: 8
  - Possible values to try: 4, 12, 16
- `strain_inds`: Number of inducing points used in the strain encoder.
  - Current value: 128
  - Possible values to try: 64, 96, 160, 192  
- `phage_inds`: Number of inducing points used in the phage encoder.
  - Current value: 64
  - Possible values to try: 32, 48, 80, 96
- `dropout`: Dropout rate applied in the classifier.
  - Current value: 0.1
  - Possible values to try: 0.0, 0.2, 0.3, 0.4
- `ln`: Whether to apply layer normalization in the set encoders, cross-attention blocks, PMA layers, and classifier.
  - Current value: True
  - Possible values to try: False

In [13]:
class SetEncoder(nn.Module):
    """
    Encodes a set of embeddings into a refined set
    using ISAB layers.
    NOTE: We do NOT pool down to a single token here.
          That happens after cross-attention.
    """
    def __init__(self, dim_input, dim_output, num_heads, num_inds, ln=False):
        super(SetEncoder, self).__init__()
        self.isab1 = ISAB(dim_input, dim_output, num_heads, num_inds, ln=ln)
        self.isab2 = ISAB(dim_output, dim_output, num_heads, num_inds, ln=ln)

    def forward(self, X):
        H = self.isab1(X)  # [B, n, d]
        H = self.isab2(H)  # [B, n, d]
        return H


class StrainPhageTransformer(nn.Module):
    """
    1) Encode strain genes and phage genes via ISAB stacks.
    2) Cross-attend them at the gene level.
    3) Pool each cross-attended set to a single vector via PMA.
    4) Concatenate and classify.
    """
    def __init__(self,
                 embedding_dim=1280,
                 num_heads=8,
                 strain_inds=128,
                 phage_inds=64,
                 dropout=0.1,
                 ln=True):
        super().__init__()

        # 1) Set Transformer encoders (no final pooling)
        self.strain_encoder = SetEncoder(
            dim_input=embedding_dim,
            dim_output=embedding_dim,
            num_heads=num_heads,
            num_inds=strain_inds,
            ln=ln
        )
        self.phage_encoder = SetEncoder(
            dim_input=embedding_dim,
            dim_output=embedding_dim,
            num_heads=num_heads,
            num_inds=phage_inds,
            ln=ln
        )

        # 2) Cross-attention blocks
        self.strain_to_phage = CrossAttention(embedding_dim, num_heads, ln=ln)
        self.phage_to_strain = CrossAttention(embedding_dim, num_heads, ln=ln)

        # 3) PMA for final pooling to a single vector
        self.strain_pma = PMA(embedding_dim, num_heads, num_seeds=1, ln=ln)
        self.phage_pma  = PMA(embedding_dim, num_heads, num_seeds=1, ln=ln)

        # 4) Final classifier
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim * 2, embedding_dim),
            nn.LayerNorm(embedding_dim) if ln else nn.Identity(),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(embedding_dim, 1)
        )

    def forward(self, strain_genes, phage_genes, return_attn=False):
        """
        strain_genes: [B, n_s, 1280]
        phage_genes:  [B, n_p, 1280]
        return_attn : if True, returns (logits, (strain_attn, phage_attn)) for interpretability
        """
        # 1) Encode each genome with set encoders
        # print("strain_genes shape:", strain_genes.shape)
        # print("phage_genes shape:", phage_genes.shape)
        strain_enc = self.strain_encoder(strain_genes)  # [B, n_s, d]
        phage_enc  = self.phage_encoder(phage_genes)    # [B, n_p, d]

        # print("strain_enc shape:", strain_enc.shape)
        # print("phage_enc shape:", phage_enc.shape)

        # 2) Cross-attention
        if return_attn:
            strain_attended, strain_attn = self.strain_to_phage(strain_enc, phage_enc, return_attn=True)
            phage_attended,  phage_attn  = self.phage_to_strain(phage_enc, strain_enc, return_attn=True)
        else:
            strain_attended = self.strain_to_phage(strain_enc, phage_enc)
            phage_attended  = self.phage_to_strain(phage_enc, strain_enc)
            strain_attn = None
            phage_attn  = None

        # 3) Pool each cross-attended set
        strain_pooled = self.strain_pma(strain_attended)  # [B, 1, d]
        phage_pooled  = self.phage_pma(phage_attended)     # [B, 1, d]

        strain_vec = strain_pooled.squeeze(1)  # [B, d]
        phage_vec  = phage_pooled.squeeze(1)   # [B, d]

        # 4) Classifier
        combined = torch.cat([strain_vec, phage_vec], dim=-1)  # [B, 2*d]
        logits   = self.classifier(combined)                   # [B, 1]

        # print("logits shape:", logits.shape)  # Add this line

        # Return attention if requested
        if return_attn:
            return logits, (strain_attn, phage_attn)
        else:
            return logits

# Block 6: Training Functions
Description: This block contains the functions for training the model, including the training loop, validation, early stopping, and visualization of training history.

## Hyperparameters and Architectural Decisions
### `train_model` function:
- `num_epochs`: Maximum number of training epochs.
  - Current value: 100
  - Possible values to try: 50, 150, 200
- `learning_rate`: Learning rate used by the optimizer.
  - Current value: 1e-4
  - Possible values to try: 1e-3, 5e-4, 1e-5
- `patience`: Number of epochs to wait for improvement before triggering early stopping.
  - Current value: 7
  - Possible values to try: 5, 10, 15
- `pos_weight_val`: Weight assigned to the positive class in the loss function to handle class imbalance.
  - Current value: Computed based on class distribution in the training data
  - Possible values to try: Adjust the computed value by a factor of 0.8, 1.2, 1.5

In [14]:
class EarlyStopping:
    """
    Early stopping to stop training when a metric (e.g., MCC) has stopped improving.
    If 'mode' is 'max', we look for metric going UP. If 'mode' is 'min', we look for metric going DOWN.
    """
    def __init__(self, patience=7, min_delta=0.0, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode  # 'max' or 'min'
        self.best_metric = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, current_metric):
        if self.best_metric is None:
            self.best_metric = current_metric
        else:
            # For 'max' mode, improvement means current_metric >= best_metric + min_delta
            if self.mode == 'max':
                if current_metric < (self.best_metric + self.min_delta):
                    self.counter += 1
                else:
                    self.best_metric = current_metric
                    self.counter = 0
            else:  # 'min' mode
                if current_metric > (self.best_metric - self.min_delta):
                    self.counter += 1
                else:
                    self.best_metric = current_metric
                    self.counter = 0

            if self.counter >= self.patience:
                self.early_stop = True

def train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in train_loader:  # Remove tqdm here
        strain_emb, phage_emb, labels = batch
        strain_emb = strain_emb.to(device)
        phage_emb = phage_emb.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(strain_emb, phage_emb)
        loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        preds = torch.sigmoid(logits).reshape(-1).detach().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    epoch_loss = total_loss / len(train_loader)
    epoch_mcc = matthews_corrcoef(
        (np.array(all_labels) > 0.5).astype(int),
        (np.array(all_preds) > 0.5).astype(int)
    )
    return epoch_loss, epoch_mcc

def validate(model, val_loader, criterion, device):
    """Validate the model on a held-out set."""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for strain_emb, phage_emb, labels in val_loader:
            strain_emb = strain_emb.to(device)
            phage_emb  = phage_emb.to(device)
            labels     = labels.to(device)

            logits = model(strain_emb, phage_emb)
            loss   = criterion(logits, labels)

            total_loss += loss.item()

            preds = torch.sigmoid(logits).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    val_loss = total_loss / len(val_loader)
    val_mcc = matthews_corrcoef(
        (np.array(all_labels) > 0.5).astype(int),
        (np.array(all_preds) > 0.5).astype(int)
    )
    return val_loss, val_mcc

def train_model(model, train_loader, val_loader,
                num_epochs=100,
                learning_rate=1e-4,
                patience=7,
                device='cuda',
                pos_weight_val=None):

    if pos_weight_val is not None:
        criterion = nn.BCEWithLogitsLoss(
            pos_weight=torch.tensor([pos_weight_val], device=device)
        )
    else:
        criterion = nn.BCEWithLogitsLoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )
    early_stopping = EarlyStopping(patience=patience, mode='max')
    history = {'train_loss': [], 'val_loss': [], 'train_mcc': [], 'val_mcc': []}

    # Calculate total steps for tqdm
    total_steps = len(train_loader) * num_epochs

    # Create a single tqdm loop for the entire training process
    pbar_epochs = tqdm(total=num_epochs, desc="Training", position=0, leave=True)

    try:
        for epoch in range(num_epochs):
            # Train and validate
            train_loss, train_mcc = train_epoch(model, train_loader, optimizer, criterion, device)
            val_loss, val_mcc = validate(model, val_loader, criterion, device)

            # Update scheduler and history
            scheduler.step(val_mcc)
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['train_mcc'].append(train_mcc)
            history['val_mcc'].append(val_mcc)

            # Update tqdm progress bar after each epoch
            pbar_epochs.update(1)  # Increment by 1 for each completed epoch
            pbar_epochs.set_postfix({
                'Epoch': epoch + 1,
                'train_loss': f'{train_loss:.3f}',
                'val_loss': f'{val_loss:.3f}',
                'train_mcc': f'{train_mcc:.3f}',
                'val_mcc': f'{val_mcc:.3f}'
            })

            # Check early stopping
            early_stopping(val_mcc)
            if early_stopping.early_stop:
                print("\nEarly stopping triggered.")
                break

    finally:
        pbar_epochs.close()  # Close tqdm progress bar

    return history

def plot_training_history(history):
    epochs = len(history['train_loss'])
    plt.figure(figsize=(12,5))

    # Loss
    plt.subplot(1,2,1)
    plt.plot(range(1, epochs+1), history['train_loss'], label='Train Loss')
    plt.plot(range(1, epochs+1), history['val_loss'],   label='Val Loss')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Loss vs. Epochs")
    plt.legend()

    # MCC
    plt.subplot(1,2,2)
    plt.plot(range(1, epochs+1), history['train_mcc'], label='Train MCC')
    plt.plot(range(1, epochs+1), history['val_mcc'],   label='Val MCC')
    plt.xlabel("Epoch")
    plt.ylabel("MCC")
    plt.title("MCC vs. Epochs")
    plt.legend()

    plt.show()

## Model Performance Evaluation

In [15]:
def predict_and_evaluate(model, data_loader, device):
    """Makes predictions and calculates performance metrics."""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for strain_emb, phage_emb, labels in data_loader:
            strain_emb = strain_emb.to(device)
            phage_emb = phage_emb.to(device)
            labels = labels.to(device)

            logits = model(strain_emb, phage_emb)  # Get raw logits
            preds = torch.sigmoid(logits).cpu().numpy()  # Apply sigmoid to get probabilities
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels).flatten()

    # Calculate metrics
    accuracy = accuracy_score(all_labels, (all_preds > 0.5).astype(int))
    precision = precision_score(all_labels, (all_preds > 0.5).astype(int))
    recall = recall_score(all_labels, (all_preds > 0.5).astype(int))
    f1 = f1_score(all_labels, (all_preds > 0.5).astype(int))
    mcc = matthews_corrcoef(all_labels, (all_preds > 0.5).astype(int))
    roc_auc = roc_auc_score(all_labels, all_preds)
    pr_auc = average_precision_score(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, (all_preds > 0.5).astype(int))

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

    return accuracy, precision, recall, f1, mcc, roc_auc, pr_auc, conf_matrix, all_preds, all_labels

def plot_confusion_matrix(conf_matrix):
    """Plots the confusion matrix using seaborn."""
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
                xticklabels=['Predicted Negative', 'Predicted Positive'],
                yticklabels=['Actual Negative', 'Actual Positive'])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

def plot_roc_curve(all_labels, all_preds):
    """Plots the ROC curve."""
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

def plot_precision_recall_curve(all_labels, all_preds):
    """Plots the precision-recall curve."""
    precision, recall, _ = precision_recall_curve(all_labels, all_preds)
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='darkorange', lw=2, label=f'PR curve (area = {pr_auc:.2f})')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    plt.show()

def get_attention_scores(model, strain_emb, phage_emb, device):
    """Gets attention scores from the model."""
    model.eval()
    with torch.no_grad():
        strain_emb = strain_emb.to(device)
        phage_emb = phage_emb.to(device)
        _, (strain_attn, phage_attn) = model(strain_emb, phage_emb, return_attn=True)
    return strain_attn, phage_attn # these are still on device, need to .cpu() if needed

# accuracy, precision, recall, f1, mcc, roc_auc, pr_auc, conf_matrix = predict_and_evaluate(model, test_loader, device)
# plot_confusion_matrix(conf_matrix)
# plot_roc_curve(all_labels, all_preds)  # Assuming you saved all_labels and all_preds in predict_and_evaluate
# plot_precision_recall_curve(all_labels, all_preds)


## Main Training Script

Description: This block contains a function for the full workflow.

In [16]:
def main(debug=False):
    # 1. Load data
    strain_embeddings = load_embeddings('/content/drive/MyDrive/Arkin/set_transformer_data/embedding_sets/strains/')
    phage_embeddings  = load_embeddings('/content/drive/MyDrive/Arkin/set_transformer_data/embedding_sets/phages/')

    # 2. Load and split interactions
    interactions_df = pd.read_csv('/content/drive/MyDrive/Arkin/set_transformer_data/EDGE_interaction_long_172_no2.csv')

    # Filter interactions_df before splitting
    strain_keys = set(strain_embeddings.keys())
    phage_keys = set(phage_embeddings.keys())
    interactions_df = interactions_df[interactions_df['strain'].isin(strain_keys) & interactions_df['phage'].isin(phage_keys)]

    # Debug mode filtering
    if debug:
        random_strains = random.sample(list(strain_keys), 30)
        random_phages = random.sample(list(phage_keys), 30)
        interactions_df = interactions_df[interactions_df['strain'].isin(random_strains) & interactions_df['phage'].isin(random_phages)]

    train_df, test_df = filter_interactions_by_strain(interactions_df)

    # 2a. Compute pos_weight from train_df
    num_pos = (train_df['interaction'] == 1).sum()
    num_neg = (train_df['interaction'] == 0).sum()
    pos_weight_val = num_neg / max(num_pos, 1)  # avoid divide-by-zero

    print(f"pos_weight_val = {pos_weight_val:.3f}")

    # 3. Create data loaders
    train_loader, test_loader = create_data_loaders(
        train_df,
        test_df,
        strain_embeddings,
        phage_embeddings,
        batch_size=1
    )

    # 4. Initialize model
    model = StrainPhageTransformer(
        embedding_dim=1280,
        num_heads=8,
        strain_inds=128,
        phage_inds=64,
        dropout=0.1,
        ln=True
    ).to(device)

    # 5. Train model, passing pos_weight_val
    history = train_model(
        model,
        train_loader,
        test_loader,
        num_epochs=100,
        learning_rate=1e-4,
        patience=7,
        device=device,
        pos_weight_val=pos_weight_val
    )

    # 6. Save model and results
    torch.save({
        'model_state_dict': model.state_dict(),
        'history': history,
    }, 'strain_phage_transformer.pt')

    # 7. Plot training history
    plot_training_history(history)

    # 6. Prediction and Evaluation within main()
    accuracy, precision, recall, f1, mcc, roc_auc, pr_auc, conf_matrix, all_preds, all_labels = predict_and_evaluate(model, test_loader, device)

    # 7. Plotting within main() (optional, but convenient)
    plot_confusion_matrix(conf_matrix)
    plot_roc_curve(all_labels, all_preds)  # Assuming you have all_labels and all_preds from predict_and_evaluate
    plot_precision_recall_curve(all_labels, all_preds)

    # Return the desired values
    return model, history, accuracy, precision, recall, f1, mcc, roc_auc, pr_auc, conf_matrix, all_preds, all_labels

In [ ]:
if __name__ == "__main__":
    model, history, accuracy, precision, recall, f1, mcc, roc_auc, pr_auc, conf_matrix, all_preds, all_labels = main(debug=True)

    # Create a dictionary to store the outputs
    outputs = {
         'model': model,
         'history': history,
         'accuracy': accuracy,
         'precision': precision,
         'recall': recall,
         'f1': f1,
         'mcc': mcc,
         'roc_auc': roc_auc,
         'pr_auc': pr_auc,
         'conf_matrix': conf_matrix,
         'all_preds': all_preds,
         'all_labels': all_labels
    }

    # Get the directory of the interaction matrix
    output_dir = '/content/drive/MyDrive/Arkin/set_transformer_data/'

    # Create a subdirectory for outputs if it doesn't exist
    output_subdir = os.path.join(output_dir, 'outputs')
    os.makedirs(output_subdir, exist_ok=True)

    # Construct the full file path for the output file
    output_file_path = os.path.join(output_subdir, 'main_function_outputs_test.pkl')  # or .joblib

    # Save the outputs
    with open(output_file_path, 'wb') as f:  # or 'wb' for joblib
        pickle.dump(outputs, f)  # or joblib.dump(outputs, f)



Parsing:  ECOR17
Parsing:  ECOR71
Parsing:  ECRC22
Parsing:  ECRC46
Parsing:  ECRC21
Parsing:  ECRC35
Parsing:  NILS34
Parsing:  NILS33
Parsing:  ECOR62
Parsing:  DEC1A
Parsing:  ECOR20
Parsing:  NILS40
Parsing:  ECOR5
Parsing:  ECOR50
Parsing:  ECOR7
Parsing:  ECOR37
Parsing:  ECRC13
Parsing:  B6A1
Parsing:  ECOR10
Parsing:  ECOR47
Parsing:  ECOR29
Parsing:  ECOR36
Parsing:  916A
Parsing:  ECOR38
Parsing:  ECOR59
Parsing:  ECOR61
Parsing:  ECRC17
Parsing:  ECRC12
Parsing:  ECOR40
Parsing:  ECOR70
Parsing:  ECOR9
Parsing:  ECOR43
Parsing:  ECOR41
Parsing:  ECOR14
Parsing:  ECOR39
Parsing:  ECOR6
Parsing:  ECOR52
Parsing:  ECOR44
Parsing:  ECOR56
Parsing:  BW25113
Parsing:  ECOR58
Parsing:  ECOR69
Parsing:  ECOR15
Parsing:  ECOR55
Parsing:  ECOR16
Parsing:  ECOR31
Parsing:  ECOR54
Parsing:  55989
Parsing:  ECOR63
Parsing:  ECOR24
Parsing:  ECOR67
Parsing:  ECOR53
Parsing:  ECOR2
Parsing:  ECOR49
Parsing:  ECOR35
Parsing:  ECRC14
Parsing:  ECOR23
Parsing:  ECRC2
Parsing:  ECRC1
Parsing: 

Training:  12%|█▏        | 12/100 [03:45<27:25, 18.70s/it, Epoch=12, train_loss=3.837, val_loss=10.079, train_mcc=0.000, val_mcc=0.000]

## Model Performance